<a href="https://colab.research.google.com/github/XuyangAbert/PyGCL_Personal_YD/blob/main/PyGCL_Prac_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/XuyangAbert/PyGCL.git

Cloning into 'PyGCL'...
remote: Enumerating objects: 1794, done.
remote: Counting objects: 100% (665/665), done.
remote: Compressing objects: 100% (295/295), done.
remote: Total 1794 (delta 554), reused 370 (delta 370), pack-reused 1129 (from 2)
Receiving objects: 100% (1794/1794), 498.54 KiB | 5.67 MiB/s, done.
Resolving deltas: 100% (1186/1186), done.


In [ ]:
# !pip install torch==2.1.2
# !pip install torch-geometric==2.2.0
!pip install torch
!pip install torch-geometric
!pip install numpy==1.24.3
# !pip install numpy
!pip install tqdm
!pip install scipy
!pip install networkx
!pip install scikit-learn
!pip install dgl==1.0.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 57.7 MB/s eta 0:00:00


In [ ]:
!pip install torch_scatter
!pip install torch_sparse

  Using cached torch_scatter-2.1.2.tar.gz (108 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for torch_scatter: filename=torch_scatter-2.1.2-cp311-cp311-linux_x86_64.whl size=3622715 sha256=a37494b93b6cff4e11ca274842574ca063a0d2e6e076560b673d3e22e6fe7541
  Stored in directory: /root/.cache/pip/wheels/b8/d4/0e/a80af2465354ea7355a2c153b11af2da739cfcf08b6c0b28e2
Successfully built torch_scatter
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch_sparse: filename=torch_sparse-0.6.18-cp311-cp311-linux_x86_64.whl size=2846215 sha256=c0fcb2861674a21e18ca4a17f2743b8d04d6c9c53d34b094f9c0b985cc4b441b
  Stored in directory: /root/.cache/pip/wheels/75/e2/1e/299c596063839303657c211f587f05591891cc6cf126d94d21
Successfully built torch_sparse


In [ ]:
import sys
sys.path.append('/content/PyGCL')

Graph Classification Tasks

In [ ]:
# import sys
# sys.path.append('/content/PyGCL')
import torch
import os.path as osp
import GCL.losses as L
import GCL.augmentors as A
import torch.nn.functional as F

from torch import nn
from tqdm import tqdm
from torch.optim import Adam
from GCL.eval import get_split, SVMEvaluator
from GCL.models import DualBranchContrast
from torch_geometric.nn import GINConv, global_add_pool
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset,Planetoid


def make_gin_conv(input_dim, out_dim):
    return GINConv(nn.Sequential(nn.Linear(input_dim, out_dim), nn.ReLU(), nn.Linear(out_dim, out_dim)))


class GConv(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(GConv, self).__init__()
        self.layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()

        for i in range(num_layers):
            if i == 0:
                self.layers.append(make_gin_conv(input_dim, hidden_dim))
            else:
                self.layers.append(make_gin_conv(hidden_dim, hidden_dim))
            self.batch_norms.append(nn.BatchNorm1d(hidden_dim))

        project_dim = hidden_dim * num_layers
        self.project = torch.nn.Sequential(
            nn.Linear(project_dim, project_dim),
            nn.ReLU(inplace=True),
            nn.Linear(project_dim, project_dim))

    def forward(self, x, edge_index, batch):
        z = x
        zs = []
        for conv, bn in zip(self.layers, self.batch_norms):
            z = conv(z, edge_index)
            z = F.relu(z)
            z = bn(z)
            zs.append(z)
        gs = [global_add_pool(z, batch) for z in zs]
        z, g = [torch.cat(x, dim=1) for x in [zs, gs]]
        return z, g


class Encoder(torch.nn.Module):
    def __init__(self, encoder, augmentor):
        super(Encoder, self).__init__()
        self.encoder = encoder
        self.augmentor = augmentor

    def forward(self, x, edge_index, batch):
        aug1, aug2 = self.augmentor
        x1, edge_index1, edge_weight1 = aug1(x, edge_index)
        x2, edge_index2, edge_weight2 = aug2(x, edge_index)
        z, g = self.encoder(x, edge_index, batch)
        z1, g1 = self.encoder(x1, edge_index1, batch)
        z2, g2 = self.encoder(x2, edge_index2, batch)
        return z, g, z1, z2, g1, g2


def train(encoder_model, contrast_model, dataloader, optimizer):
    encoder_model.train()
    epoch_loss = 0
    for data in dataloader:
        data = data.to('cuda')
        optimizer.zero_grad()

        if data.x is None:
            num_nodes = data.batch.size(0)
            data.x = torch.ones((num_nodes, 1), dtype=torch.float32, device=data.batch.device)

        _, _, _, _, g1, g2 = encoder_model(data.x, data.edge_index, data.batch)
        g1, g2 = [encoder_model.encoder.project(g) for g in [g1, g2]]
        loss = contrast_model(g1=g1, g2=g2, batch=data.batch)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
    return epoch_loss


def test(encoder_model, dataloader):
    encoder_model.eval()
    x = []
    y = []
    for data in dataloader:
        data = data.to('cuda')
        if data.x is None:
            num_nodes = data.batch.size(0)
            data.x = torch.ones((num_nodes, 1), dtype=torch.float32, device=data.batch.device)
        _, g, _, _, _, _ = encoder_model(data.x, data.edge_index, data.batch)
        x.append(g)
        y.append(data.y)
    x = torch.cat(x, dim=0)
    y = torch.cat(y, dim=0)

    split = get_split(num_samples=x.size()[0], train_ratio=0.6, test_ratio=0.2)
    result = SVMEvaluator(linear=True)(x, y, split)
    return result

Example Codes for Graph Classification

In [ ]:
def graph_classification():
    device = torch.device('cuda')
    path = osp.join(osp.expanduser('~'), 'datasets')
    dataset = TUDataset(path, name='PTC_MR')
    dataloader = DataLoader(dataset, batch_size=128)
    input_dim = max(dataset.num_features, 1)

    aug1 = A.Identity()
    aug2 = A.RandomChoice([A.RWSampling(num_seeds=1000, walk_length=10),
                           A.NodeDropping(pn=0.1),
                           A.FeatureMasking(pf=0.1),
                           A.EdgeRemoving(pe=0.1)], 1)
    gconv = GConv(input_dim=input_dim, hidden_dim=32, num_layers=2).to(device)
    encoder_model = Encoder(encoder=gconv, augmentor=(aug1, aug2)).to(device)
    contrast_model = DualBranchContrast(loss=L.InfoNCE(tau=0.2), mode='G2G').to(device)

    optimizer = Adam(encoder_model.parameters(), lr=0.00001)
    test_results = []

    with tqdm(total=100, desc='(T)') as pbar:
        for epoch in range(1, 101):
            loss = train(encoder_model, contrast_model, dataloader, optimizer)
            if epoch % 5 == 0:
                test_result = test(encoder_model, dataloader)
                test_results.append([epoch,
                                     test_result['micro_f1'],
                                     test_result['macro_f1'],
                                     loss])
            # pbar.set_postfix({'loss': loss})
            # pbar.update()
    return test_results

    # test_result = test(encoder_model, dataloader)
    # print(f'(E): Best test F1Mi={test_result["micro_f1"]:.4f}, F1Ma={test_result["macro_f1"]:.4f}')


if __name__ == '__main__':
    test_results = graph_classification()


Modify the codes from MVGL_node.py and GrapgCL.py to make it suitable for both the node-level and graph-level tasks.

In [ ]:
import torch
import os.path as osp
import GCL.losses as L
import GCL.augmentors as A
import torch.nn.functional as F
import torch_geometric.transforms as T

from torch import nn
from tqdm import tqdm
from torch.optim import Adam, SGD
from GCL.eval import get_split, SVMEvaluator
from GCL.models import DualBranchContrast
from torch_geometric.nn import GINConv, global_add_pool
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset,Planetoid


def make_gin_conv(input_dim, out_dim):
    return GINConv(nn.Sequential(nn.Linear(input_dim, out_dim), nn.ReLU(), nn.Linear(out_dim, out_dim)))


class GConv(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(GConv, self).__init__()
        self.layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()

        for i in range(num_layers):
            if i == 0:
                self.layers.append(make_gin_conv(input_dim, hidden_dim))
            else:
                self.layers.append(make_gin_conv(hidden_dim, hidden_dim))
            self.batch_norms.append(nn.BatchNorm1d(hidden_dim))

        project_dim = hidden_dim * num_layers
        self.project = torch.nn.Sequential(
            nn.Linear(project_dim, project_dim),
            nn.ReLU(inplace=True),
            nn.Linear(project_dim, project_dim))

    def forward(self, x, edge_index, batch):
        z = x
        zs = []
        for conv, bn in zip(self.layers, self.batch_norms):
            z = conv(z, edge_index)
            z = F.relu(z)
            z = bn(z)
            zs.append(z)
        z = torch.cat(zs, dim=1)
        z = self.project(z)
        return z


class Encoder(torch.nn.Module):
    def __init__(self, encoder, augmentor):
        super(Encoder, self).__init__()
        self.encoder = encoder
        self.augmentor = augmentor

    def forward(self, x, edge_index, batch):
        aug1, aug2 = self.augmentor
        x1, edge_index1, edge_weight1 = aug1(x, edge_index)
        x2, edge_index2, edge_weight2 = aug2(x, edge_index)
        z = self.encoder(x, edge_index, batch)
        z1 = self.encoder(x1, edge_index1, batch)
        z2 = self.encoder(x2, edge_index2, batch)
        return z, z1, z2

def train_node(encoder_model, contrast_model, data, optimizer):
  encoder_model.train()
  optimizer.zero_grad()
  z, z1, z2 = encoder_model(data.x, data.edge_index, data.batch)
  loss = contrast_model(h1=z1, h2=z2, batch=data.batch)
  loss.backward()
  optimizer.step()
  return loss.item()

def test_node(encoder_model, data):
  encoder_model.eval()
  z, _, _ = encoder_model(data.x, data.edge_index, data.batch)
  split = get_split(num_samples=z.size()[0], train_ratio=0.6, test_ratio=0.2)
  result = SVMEvaluator(linear=True)(z, data.y, split)
  return result

def node_classification(option):
    device = torch.device('cuda')
    path = osp.join(osp.expanduser('~'), 'datasets')
    dataset = Planetoid(path, name='Cora', transform=T.NormalizeFeatures())
    data = dataset[0].to(device)
    input_dim = max(dataset.num_features, 1)

    ## Choose for different sparsification methods for augmentation ##
    # aug1 = A.Identity()
    aug1 = A.PPRDiffusion(alpha = 0.2)
    if option == 'fosr':
      aug2 = A.FOSR(max_iterations = 20)
    elif option == 'proxydelmin':
      aug2 = A.PROXYDELMIN(max_iterations = 20, seed = 4438)
    elif option == 'proxydelmax':
      aug2 = A.PROXYDELMAX(max_iterations = 20, seed = 4438)
    elif option == 'sdrf':
      aug2 = A.SDRF(max_iterations = 20)
    else:
      aug2 = A.Identity()
    gconv = GConv(input_dim=input_dim, hidden_dim=128, num_layers=2).to(device)
    encoder_model = Encoder(encoder=gconv, augmentor=(aug1, aug2)).to(device)
    contrast_model = DualBranchContrast(loss=L.InfoNCE(tau=0.2), mode='L2L').to(device)

    optimizer = Adam(encoder_model.parameters(), lr=0.01)
    optimizer = SGD(encoder_model.parameters(), lr=0.01)
    test_results = []

    with tqdm(total=100, desc='(T)') as pbar:
        for epoch in range(1, 101):
            loss = train_node(encoder_model, contrast_model, data, optimizer)
            if epoch % 5 == 0:
                test_result = test_node(encoder_model, data)
                test_results.append([epoch,
                                     test_result['micro_f1'],
                                     test_result['macro_f1'],
                                     loss])
            pbar.set_postfix({'loss': loss})
            pbar.update()
    return test_results


In [ ]:
res1 = node_classification('fosr')
res2 = node_classification('proxydelmin')
res3 = node_classification('proxydelmax')
res4 = node_classification('sdrf')
# res5 = node_classification()
res5 = node_classification('other')

(T):   0%|          | 0/100 [00:00<?, ?it/s]

No more edges can be modified to increase the spectral gap.


(T):   1%|          | 1/100 [00:10<16:43, 10.13s/it, loss=7.77]

No more edges can be modified to increase the spectral gap.


(T):   2%|▏         | 2/100 [00:20<16:20, 10.00s/it, loss=7.67]

...

(T):  98%|█████████▊| 98/100 [12:05<00:12,  6.40s/it, loss=5.76]

Number of edges modified 5


(T):  99%|█████████▉| 99/100 [12:09<00:05,  5.74s/it, loss=5.75]

Number of edges modified 5
Number of edges modified 5


(T): 100%|██████████| 100/100 [03:35<00:00,  2.15s/it, loss=5.19]


In [ ]:
import pandas as pd
import numpy as np
micro_f1 = pd.DataFrame({"Epoch": np.array(res1)[:, 0],
                         "FOSR + PPRDIFF": np.array(res1)[:, 1],
                         "PROXDELMIN + PPRDIFF": np.array(res2)[:, 1],
                         "PROXDELMAX + PPRDIFF": np.array(res3)[:, 1],
                         "SDRF": np.array(res4)[:, 1],
                         "IDENTITY + PPRDIFF": np.array(res5)[:, 1]})
macro_f1 = pd.DataFrame({"Epoch": np.array(res1)[:, 0],
                         "FOSR + PPRDIFF": np.array(res1)[:, 2],
                         "PROXDELMIN + PPRDIFF": np.array(res2)[:, 2],
                         "PROXDELMAX + PPRDIFF": np.array(res3)[:, 2],
                         "SDRF": np.array(res4)[:, 2],
                         "IDENTITY + PPRDIFF": np.array(res5)[:, 2]})

micro_f1.to_csv('micro_f1_ADAW_001.csv')
macro_f1.to_csv('macro_f1_ADAW_001.csv')